In [1]:
import pandas as pd
import numpy as np
import zipfile
import os

In [ ]:
os.makedirs('./enem2019', exist_ok=True)

In [ ]:
myzip = zipfile.ZipFile("./microdados_enem_2019.zip")
myzip.extractall("./enem2019/")

In [2]:
filename = "./enem2019/DADOS/MICRODADOS_ENEM_2019.csv"

In [92]:
import chardet
with open(filename, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [4]:
# determinana os tipos das colunas para uma amostra para posterior leitura da base completa.
sample = pd.read_csv(filename, low_memory=False, nrows=1000, encoding="ISO-8859-1")
# print("Tipos das colunas do DF")
# print(dict(sample.dtypes))
types = sample.dtypes
types = types.apply(str)
dict_types = types.to_dict()
dict_types

{'NU_INSCRICAO;NU_ANO;CO_MUNICIPIO_RESIDENCIA;NO_MUNICIPIO_RESIDENCIA;CO_UF_RESIDENCIA;SG_UF_RESIDENCIA;NU_IDADE;TP_SEXO;TP_ESTADO_CIVIL;TP_COR_RACA;TP_NACIONALIDADE;CO_MUNICIPIO_NASCIMENTO;NO_MUNICIPIO_NASCIMENTO;CO_UF_NASCIMENTO;SG_UF_NASCIMENTO;TP_ST_CONCLUSAO;TP_ANO_CONCLUIU;TP_ESCOLA;TP_ENSINO;IN_TREINEIRO;CO_ESCOLA;CO_MUNICIPIO_ESC;NO_MUNICIPIO_ESC;CO_UF_ESC;SG_UF_ESC;TP_DEPENDENCIA_ADM_ESC;TP_LOCALIZACAO_ESC;TP_SIT_FUNC_ESC;IN_BAIXA_VISAO;IN_CEGUEIRA;IN_SURDEZ;IN_DEFICIENCIA_AUDITIVA;IN_SURDO_CEGUEIRA;IN_DEFICIENCIA_FISICA;IN_DEFICIENCIA_MENTAL;IN_DEFICIT_ATENCAO;IN_DISLEXIA;IN_DISCALCULIA;IN_AUTISMO;IN_VISAO_MONOCULAR;IN_OUTRA_DEF;IN_GESTANTE;IN_LACTANTE;IN_IDOSO;IN_ESTUDA_CLASSE_HOSPITALAR;IN_SEM_RECURSO;IN_BRAILLE;IN_AMPLIADA_24;IN_AMPLIADA_18;IN_LEDOR;IN_ACESSO;IN_TRANSCRICAO;IN_LIBRAS;IN_TEMPO_ADICIONAL;IN_LEITURA_LABIAL;IN_MESA_CADEIRA_RODAS;IN_MESA_CADEIRA_SEPARADA;IN_APOIO_PERNA;IN_GUIA_INTERPRETE;IN_COMPUTADOR;IN_CADEIRA_ESPECIAL;IN_CADEIRA_CANHOTO;IN_CADEIRA_ACOLCHOADA

In [19]:
# realiza uma leitura em partes/blocos (batches) apenas para alunos de MG
#encoding="ISO-8859-1" ou encoding="Latin1"?
array_df = []
for chunk in pd.read_csv(filename, encoding='Latin1', sep=';', chunksize = 10000):
    temp_df = chunk.loc[chunk['SG_UF_ESC'] == 'MG']
    array_df.append(temp_df)

df = pd.concat(array_df, ignore_index=True)

In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108173 entries, 0 to 108172
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float64(24), int64(71), object(41)
memory usage: 360.8 MB


In [25]:
#Ajusta tipos dos atributos

ints = df.select_dtypes(include=['int64','int32','int16']).columns
df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')

floats = df.select_dtypes(include=['float']).columns
df[floats] = df[floats].apply(pd.to_numeric, downcast='float')

In [26]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108173 entries, 0 to 108172
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float32(24), int16(1), int32(2), int64(1), int8(67), object(41)
memory usage: 301.1 MB


In [27]:
#Ajusta tipos dos atributos que são objects para category
objects = df.select_dtypes('object').columns
df[objects] = df[objects].apply(lambda x: x.astype('category'))

In [28]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108173 entries, 0 to 108172
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: category(41), float32(24), int16(1), int32(2), int64(1), int8(67)
memory usage: 72.0 MB


In [29]:
#Qual é a média da nota em matemática de todos os alunos mineiros?
df['NU_NOTA_MT'].mean()

547.8761596679688

In [30]:
# Qual é a média da nota em Linguagens e Códigos de todos os alunos mineiros?
df['NU_NOTA_LC'].mean()

527.4235229492188

In [54]:
#Qual é a média da nota em Ciências Humanas dos alunos do sexo FEMININO mineiros?
filtered_df = df[df.TP_SEXO.eq('F')]
filtered_df['NU_NOTA_CH'].mean()

509.9061584472656

In [57]:
#Qual é a média da nota em Ciências Humanas dos alunos do sexo MASCULINO?
filtered_df = df[df.TP_SEXO.eq('M')]
filtered_df['NU_NOTA_CH'].mean()

525.7894897460938

In [63]:
# Qual é a média da nota em Matemática dos alunos do sexo FEMININO que moram na cidade de Montes Claros?
filtered_df = df[df.TP_SEXO.eq('F')]
filtered_df = filtered_df[filtered_df.NO_MUNICIPIO_RESIDENCIA.eq('Montes Claros')]
filtered_df['NU_NOTA_MT'].mean()

528.9071044921875

In [73]:
#Qual é a média da nota em Matemática dos alunos do município de Sabará que possuem TV por assinatura na residência?
filtered_df = df[df.NO_MUNICIPIO_RESIDENCIA.eq('Sabará')]
filtered_df = filtered_df[filtered_df.Q021.eq('B')]
filtered_df['NU_NOTA_MT'].mean()

545.0728149414062

In [76]:
#Qual é a média da nota em Ciências Humanas dos alunos mineiros que possuem dois fornos micro-ondas em casa?
filtered_df = df[df.Q016.eq('C')]
filtered_df['NU_NOTA_CH'].mean()

554.8231201171875

In [77]:
#Qual é a nota média em Matemática dos alunos mineiros cuja mãe completou a pós-graduação?
filtered_df = df[df.Q002.eq('G')]
filtered_df['NU_NOTA_MT'].mean()

626.6366577148438

In [85]:
#Qual é a nota média em Matemática dos alunos de Belo Horizonte e de Conselheiro Lafaiete?
array_df_ = []
filtered_df_BH = df[df.NO_MUNICIPIO_RESIDENCIA.eq('Belo Horizonte')]
filtered_df_CL = df[df.NO_MUNICIPIO_RESIDENCIA.eq('Conselheiro Lafaiete')]
array_df_.append(filtered_df_BH)
array_df_.append(filtered_df_CL)

df_concat = pd.concat(array_df_, ignore_index=True)
df_concat['NU_NOTA_MT'].mean()

585.5897216796875

In [88]:
#Qual é a nota média em Ciências Humanas dos alunos mineiros que moram sozinhos?
filtered_df = df[df.Q005.eq(1)]
filtered_df['NU_NOTA_CH'].mean()

512.2308959960938

In [89]:
# Qual é a nota média em Ciências Humanas dos alunos mineiros cujo pai completou pós-graduação e possuem renda familiar entre R$ 8.982,01 e R$ 9.980,00.
filtered_df = df[(df['Q001'] == 'G') & (df['Q006'] == 'M' )]
filtered_df['NU_NOTA_CH'].mean()

597.1246337890625

In [90]:
# Qual é a nota média em Matemática dos alunos do sexo Feminino que moram em Lavras e escolheram “Espanhol” como língua estrangeira?
filtered_df = df[(df['NO_MUNICIPIO_RESIDENCIA'] == 'Lavras') & (df['TP_SEXO'] == 'F') & (df['TP_LINGUA'] == 1 )]
filtered_df['NU_NOTA_MT'].mean()

512.4329833984375

In [91]:
# Qual é a nota média em Matemática dos alunos do sexo Masculino que moram em Ouro Preto?
filtered_df = df[(df['NO_MUNICIPIO_RESIDENCIA'] == 'Ouro Preto') & (df['TP_SEXO'] == 'M')]
filtered_df['NU_NOTA_MT'].mean()

544.0599365234375

In [93]:
#Qual é a nota média em Ciências Humanas dos alunos surdos?
filtered_df = df[(df['IN_SURDEZ'] == 1)]
filtered_df['NU_NOTA_CH'].mean()

432.412109375

In [95]:
# Qual é a nota média em Matemática dos alunos do sexo FEMININO, que moram em Belo Horizonte, Sabará, Nova Lima e Betim e possuem dislexia?
filtered_df = df[df.TP_SEXO.eq('F')]
array_df_ = []
filtered_df_BH = filtered_df[filtered_df.NO_MUNICIPIO_RESIDENCIA.eq('Belo Horizonte')]
filtered_df_CL = filtered_df[filtered_df.NO_MUNICIPIO_RESIDENCIA.eq('Sabará')]
filtered_df_NL = filtered_df[filtered_df.NO_MUNICIPIO_RESIDENCIA.eq('Nova Lima')]
filtered_df_B = filtered_df[filtered_df.NO_MUNICIPIO_RESIDENCIA.eq('Betim')]

array_df_.append(filtered_df_BH)
array_df_.append(filtered_df_CL)
array_df_.append(filtered_df_NL)
array_df_.append(filtered_df_B)


df_concat = pd.concat(array_df_, ignore_index=True)
df_concat['NU_NOTA_MT'].mean()

556.9428100585938

In [80]:
import gc

gc.collect()

22